# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os, pdb
import sys
# Google developer API key
sys.path.append('../')
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
df = pd.read_csv('./output_data/clean_city_data.csv')
df.head()

,city,temp_max,humidity,cloudiness,url,lat,lon,date,wind_speed,country
0,ribeira grande,69.80,52,40,http://api.openweathermap.org/data/2.5/weather...,38.52,-28.70,1603815966,6.93,PT
1,codrington,46.67,93,19,http://api.openweathermap.org/data/2.5/weather...,-38.27,141.97,1603815958,6.76,AU
2,ruidoso,18.00,92,75,http://api.openweathermap.org/data/2.5/weather...,33.33,-105.67,1603815966,10.29,US
3,kamachumu,75.20,69,20,http://api.openweathermap.org/data/2.5/weather...,-1.62,31.62,1603815966,2.28,TZ
4,saint-philippe,77.00,69,90,http://api.openweathermap.org/data/2.5/weather...,-21.36,55.77,1603815966,2.24,RE


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
import gmaps, gmaps.geojson_geometries

In [29]:
def google_plot(df,weighting,locations):
    
    
    
    # Plot Heatmap
    fig2 = gmaps.figure(display_toolbar=True,
                        display_errors=True,
                        zoom_level=3,
                        center=(33.7490, -84.3880) #Atlanta Georgia
                       )
    figure_layout2 = {
        'width': '2000px',
        'height': '2000px',
        'border': '1px solid black',
        'padding': '1px'
    }
    gmaps.figure(layout=figure_layout2)

    # Create heat layer
    heat_layer = gmaps.heatmap_layer(locations.values, 
                                     weights=weighting, 
                                     dissipating=True, 
                                     max_intensity=df['humidity'].max()/1,
                                     point_radius=15,
                                     opacity=0.95)
    # Add layer
    fig2.add_layer(heat_layer)   
    return fig2

In [30]:
google_plot(df,weighting = df["humidity"],locations = df[["lat", "lon"]])

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
df = df.dropna(subset=['temp_max','wind_speed','cloudiness'])
df_ideal_weather = df[(df['temp_max']<=80) & (df['wind_speed']<=10) & (df['cloudiness']==0)]
google_plot(df_ideal_weather,weighting = df_ideal_weather["humidity"],locations = df_ideal_weather[["lat", "lon"]])

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
#function to return country using latitude and longitude
def nearest_hotel(row):#https://stackoverflow.com/questions/20169467/how-to-convert-from-longitude-and-latitude-to-country-or-city 
    params = {
        "key": gkey,
        "rankby": "distance",
        "type" : "lodging",
        "location": str(row['lat']) + ',' + str(row['lon'])   #"39.952583,-75.16522"
         }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json() 
    #print(requests.get(base_url, params=params).url)
    hotel_dict={}
    try:
        hotel = response['results'][0]['name']
        city = response['results'][0]['plus_code']['compound_code'].split(',')[1]
        country = response['results'][0]['plus_code']['compound_code'].split(',')[2]
        lat = response['results'][0]['geometry']['location']['lat']
        lng = response['results'][0]['geometry']['location']['lng']
    except:
        hotel = ''
        city = ''
        country = ''
        lat = ''
        lng = ''
    #pdb.set_trace()

    return(hotel,city,country,lat,lng) 

In [80]:
hotel_df = pd.DataFrame()
#hotel_df['Hotel Name'] = 0
hotel_list=[]
city_list=[]
country_list=[]
lat_list=[]
lng_list=[]
for index, row in df_ideal_weather.iterrows():
    hotel,city,country,lat,lng = nearest_hotel(row)
    hotel_list.append(hotel)
    city_list.append(city)
    country_list.append(country)
    lat_list.append(lat)
    lng_list.append(lng)

hotel_df['Hotel Name'] = hotel_list
hotel_df['City'] = city_list
hotel_df['Country'] = country_list
hotel_df['Lat'] = lat_list
hotel_df['Lng'] = lng_list

#Numerous places seem to not have valdi hotels around them, and thus they must be dropped
hotel_df.replace('',np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,ABC Girls' Hostel,Meghalaya,India,25.521135,90.222131
3,"Motel Bar Hut ,, ,,",Chukotka Autonomous Okrug,Russia,64.736165,177.503925
4,VEER Teja hostel,Rajasthan,India,27.150475,74.850034
8,Hotel Isabeles,Sonora,Mexico,30.707221,-111.825727
14,"Reabilitatsionnyy Tsentr ""Ramenskoye""",Moscow Oblast,Russia,55.629882,38.121523
15,بافت خیابان طالقانی غربی ساختمان مادر,Kerman Province,Iran,29.232778,56.607053
16,Apartment on Gornyakov,Krasnoyarsk Krai,Russia,69.487933,88.393979
18,Aksay Meshiti,Moscow Oblast,Russia,55.750800,37.997488
21,دفتر پیشخوان دولت جمال زهی (خالد),Sistan and Baluchestan Province,Iran,28.217111,61.216804
22,Hotel Jardines Baja,Baja California,Mexico,30.486750,-115.938713


In [81]:
if path.exists('./output_data/hotel_df.csv'):
    print('File already exists')
    df = pd.read_csv('./output_data/hotel_df.csv')
else:
    print('Creating new file')
    df.to_csv('./output_data/hotel_df.csv', index=False)

In [83]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [91]:
def google_plot_with_markers(df_weather,weather_weighting,weather_locations,hotel_info,marker_locations):
    # Plot Heatmap
    fig2 = gmaps.figure(display_toolbar=True,
                        display_errors=True,
                        zoom_level=2,
                        center=(33.7490, -84.3880) #Atlanta Georgia
                       )
    figure_layout2 = {
        'width': '2000px',
        'height': '2000px',
        'border': '1px solid black',
        'padding': '1px'
    }
    gmaps.figure(layout=figure_layout2)

    # Create heat layer
    heat_layer = gmaps.heatmap_layer(weather_locations.values, 
                                     weights=weather_weighting, 
                                     dissipating=True, 
                                     max_intensity=df['humidity'].max()/1,
                                     point_radius=15,
                                     opacity=0.95)
    # Add layer
    fig2.add_layer(heat_layer)   
    #Add Marker Layer!
    marker_layer = gmaps.marker_layer(marker_locations,info_box_content=hotel_info)
    # Display figure
    fig2.add_layer(marker_layer)

    return fig2

In [92]:
# Add marker layer ontop of heat map


# Display figure
google_plot_with_markers(df_ideal_weather,weather_weighting=df_ideal_weather["humidity"],weather_locations=df_ideal_weather[["lat", "lon"]], hotel_info=hotel_info,marker_locations=locations           )

Figure(layout=FigureLayout(height='420px'))